In [1]:
#load the data
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('./dataset/training_set',target_size=(50,50),batch_size=32,class_mode='binary')


Using TensorFlow backend.


Found 8000 images belonging to 2 classes.


In [2]:
#确认输入数据标签
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [13]:
#set up the cnn model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = Sequential()
#卷积层
model.add(Conv2D(32,(3,3),input_shape=(50,50,3),activation='relu'))
#池化层
model.add(MaxPool2D(pool_size=(2,2)))
#卷积层
model.add(Conv2D(32,(3,3),activation='relu'))
#池化层
model.add(MaxPool2D(pool_size=(2,2)))
#flattening layer
model.add(Flatten())
#FC layer
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [14]:
#configure the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3872)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               495744    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

![cnn_structure](structure.jpg)

In [16]:
#train the model
model.fit_generator(training_set,epochs=20)

Epoch 1/20
250/250 [==============================] - 43s 170ms/step - loss: 0.6445 - accuracy: 0.6179
Epoch 2/20
250/250 [==============================] - 46s 184ms/step - loss: 0.5424 - accuracy: 0.7254
Epoch 3/20
250/250 [==============================] - 56s 223ms/step - loss: 0.4877 - accuracy: 0.7653
Epoch 4/20
250/250 [==============================] - 58s 233ms/step - loss: 0.4472 - accuracy: 0.7900
Epoch 5/20
250/250 [==============================] - 51s 204ms/step - loss: 0.4161 - accuracy: 0.8096
Epoch 6/20
250/250 [==============================] - 48s 193ms/step - loss: 0.3653 - accuracy: 0.8304
Epoch 7/20
250/250 [==============================] - 52s 209ms/step - loss: 0.3281 - accuracy: 0.8572
Epoch 8/20
250/250 [==============================] - 52s 209ms/step - loss: 0.2778 - accuracy: 0.8840s - loss: 0.2755 - accuracy -
Epoch 9/20
250/250 [==============================] - 50s 199ms/step - loss: 0.2204 - accuracy: 0.9056
Epoch 10/20
250/250 [=======================

In [17]:
#accuracy on the training data
accuracy_train = model.evaluate_generator(training_set)
print(accuracy_train)

[0.00042246864177286625, 1.0]


In [18]:
#accuracy on the test data
test_set = train_datagen.flow_from_directory('./dataset/test_set',target_size=(50,50),batch_size=32,class_mode='binary')

accuracy_test = model.evaluate_generator(test_set)
print(accuracy_test)

Found 2000 images belonging to 2 classes.
[0.6088150143623352, 0.7599999904632568]


In [19]:
#load single image
from keras.preprocessing.image import load_img, img_to_array
pic_dog = 'dog.jpg'
pic_dog = load_img(pic_dog,target_size=(50,50))
pic_dog = img_to_array(pic_dog)
pic_dog = pic_dog/255
pic_dog = pic_dog.reshape(1,50,50,3)
result = model.predict_classes(pic_dog)
print(result)

[[1]]


In [20]:
pic_cat = 'cat1.jpg'
pic_cat = load_img(pic_cat,target_size=(50,50))
pic_cat = img_to_array(pic_cat)
pic_cat = pic_cat/255
pic_cat = pic_cat.reshape(1,50,50,3)
result = model.predict_classes(pic_cat)
print(result)

[[0]]


In [21]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [22]:
# make prediction on multiple images
import matplotlib as mlp
font2 = {'family' : 'SimHei',
'weight' : 'normal',
'size'   : 20,
}
mlp.rcParams['font.family'] = 'SimHei'
mlp.rcParams['axes.unicode_minus'] = False
from matplotlib import pyplot as plt
from matplotlib.image import imread
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
a = [i for i in range(1,10)]
fig = plt.figure(figsize=(10,10))
for i in a:
    img_name = str(i)+'.jpg'
    img_ori = load_img(img_name, target_size=(50, 50))
    img = img_to_array(img_ori)
    img = img.astype('float32')/255
    img = img.reshape(1,50,50,3)
    result = model.predict_classes(img)
    img_ori = load_img(img_name, target_size=(250, 250))
    plt.subplot(3,3,i)
    plt.imshow(img_ori)
    plt.title('预测为：狗狗' if result[0][0] == 1 else '预测为：猫咪')
plt.show()